# Week 1 assignment: Linear regression using Python
The marketing department of Turtle Games prefers Python for data analysis. As you are fluent in Python, they asked you to assist with data analysis of social media data. The marketing department wants to better understand how users accumulate loyalty points. Therefore, you need to investigate the possible relationships between the loyalty points, age, remuneration, and spending scores. Note that you will use this data set in future modules as well and it is, therefore, strongly encouraged to first clean the data as per provided guidelines and then save a copy of the clean data for future use.

## Instructions
1. Load and explore the data.
    1. Create a new DataFrame (e.g. reviews).
    2. Sense-check the DataFrame.
    3. Determine if there are any missing values in the DataFrame.
    4. Create a summary of the descriptive statistics.
2. Remove redundant columns (`language` and `platform`).
3. Change column headings to names that are easier to reference (e.g. `renumeration` and `spending_score`).
4. Save a copy of the clean DataFrame as a CSV file. Import the file to sense-check.
5. Use linear regression and goodness of fit metrics to evaluate possible linear relationships between loyalty points and age/renumeration/spending scores to determine whether these can be used to predict the loyalty points.
    1. Specify the independent and dependent variables.
    2. Create the OLS model.
    3. Extract the estimated parameters, standard errors, and predicted values.
    4. Generate the regression table based on the X coefficient and constant values.
    5. Plot the linear regression and add a regression line.
6. Include your insights and observations.

## 1. Load and explore the data

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
# Load the CSV file(s) as reviews.


# View the DataFrame.

In [ ]:
# Any missing values?

In [ ]:
# Explore the data.

In [ ]:
# Basic descriptive statistics.

## 2. Drop Columns

In [ ]:
# Drop unnecessary columns.


# View column names.

## 3. Rename Columns

In [ ]:
# Rename the column headers.


# View column names.

## 4. Save the dataFrame as .CSV file

In [ ]:
# Create a CSV file as output.

In [ ]:
# Import new CSV file with Pandas.


# View DataFrame.

## 5.Linear Regression

### 5a. spending vs. loyalty

In [ ]:
# Define independent variable.


# Define dependent variable.


# Create model and print summary of metrics.

In [ ]:
# Extract the estimated parameters.


# Extract the standard errors.


# Extract the predicted values.

In [ ]:
# Set the X coefficient and the constant to generate the regression table.


# View the output.

In [ ]:
# Plot the graph with a regression line.

### 5b. renumeration loyalty

In [ ]:
# Define independent variable.


# Define dependent variable.


# Create model and print summary of metrics.

In [ ]:
# Extract the estimated parameters.


# Extract the standard errors.


# Extract the predicted values.


In [ ]:
# Set the the X coefficient and the constant to generate the regression table.


# View the output.

In [ ]:
# Plot graph with regression line.

### 5c. age vs. loyalty

In [ ]:
# Define independent variable.


# Define dependent variable.


# Create model and print summary of metrics.

In [ ]:
# Extract the estimated parameters.


# Extract the standard errors.


# Extract the predicted values.

In [ ]:
# Set the X coefficient and the constant to generate the regression table.


# View the output.

In [ ]:
# Plot graph with regression line.

## 6. Observations & Insights

_insert observations here..._